In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

In [2]:
import torch
import gc
from diffusers import StableDiffusionXLPipeline, DiffusionPipeline, DPMSolverMultistepScheduler
from transformers import CLIPTokenizer
from gen_prompt import gen_prompt

/home/j-i12b106/.conda/envs/diffusion/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
 style_lora_models = {
    "watercolor": "ostris/watercolor_style_lora_sdxl",
    "embroidery": "ostris/embroidery_style_lora_sdxl",
    "pixel_art": "artificialguybr/PixelArtRedmond",
    "linear_manga": "artificialguybr/LineAniRedmond-LinearMangaSDXL-V2",
    "studio_ghibli": "artificialguybr/StudioGhibli.Redmond-V2",
    "3d_style": "artificialguybr/3DRedmond-V1",
    "tshirt_design": "artificialguybr/TshirtDesignRedmond-V2",
    "storybook": "artificialguybr/StoryBookRedmond-V2",
    "cute_cartoon": "artificialguybr/CuteCartoonRedmond-V2"
}

In [4]:
IMG_PATH = "./result_img"

loaded_adapters = set()

def gen_image(pipe, style, prompt, negative_prompt='', save=False):
    lora_model_id = style_lora_models[style]
    model_name = lora_model_id.split('/')[-1]
    if style in loaded_adapters:
        print(f"⚠️ Adapter '{style}' is already loaded. Skipping duplicate load.")
    else:
        pipe.load_lora_weights(lora_model_id, adapter_name=style)
        loaded_adapters.add(style)
        print(f"✅ Loaded Adapter: {style}")
        # LoRA 모델 적용 (PEFT 백엔드 사용)
    
    image = pipe(
        prompt,
        negative_prompt=negative_prompt,
        num_inference_steps=30,
    ).images[0]
    
    # 이미지 저장 및 출력
    if save:
        image.save(os.path.join(IMG_PATH, f"{style}-{model_name}") + '.png')
    return image

def gen_image_pipline(genre, style, title, worldview, synopsis, characters, keywords):
    print("Generate Prompt...")
    prompt, negative_prompt = gen_prompt(genre, style, title, worldview, synopsis, characters, keywords)
    print("Generate Image...")
    image = gen_image(pipe, style, prompt, negative_prompt, save=True)

    image.show()
    

In [5]:
genre = '일상'
title = '여름 도둑'
worldview = '2025년 한국의 학교'
synopsis = '학교를 다니는 일상'
characters = 'a man'
keywords = '아련한, 즐거운, 행복한'

In [6]:
# base_model_id = "cagliostrolab/animagine-xl-4.0"
# base_model_id = "digiplay/AnalogMadness-realistic-model-v7"
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
# pipe = StableDiffusionXLPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe.to("cuda")

# PEFT 백엔드 활성화
pipe.enable_xformers_memory_efficient_attention()  # 메모리 최적화
pipe.enable_model_cpu_offload()  # CPU 오프로딩

Loading pipeline components...: 100%|██████████| 7/7 [00:07<00:00,  1.02s/it]


In [ ]:
# test all models
for style in style_lora_models:
    print(f"Generate image use {style}...")
    gen_image_pipline(genre, style, title, worldview, synopsis, characters, keywords)
    gc.collect()

Generate image use watercolor...
Generate Prompt...
Generate Image...


Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['palette )']


✅ Loaded Adapter: watercolor


Token indices sequence length is longer than the specified maximum sequence length for this model (79 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['palette )']
 97%|█████████▋| 29/30 [00:36<00:00,  1.23it/s]